In [68]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, InputLayer, Dropout, BatchNormalization, Activation, Conv2D, MaxPooling2D

In [69]:
image_width, image_height = 224, 224
num_classes = 4

In [70]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))

In [71]:
for layer in vgg.layers:
    layer.trainable = False

In [72]:
def build_model(pretrained):
    ft = Flatten()(pretrained.output)
    d1 = Dense(2048, activation='relu')(ft)
    d2 = Dense(1024, activation='relu')(d1)
    d3 = Dense(512, activation='relu')(d2)
    d4 = Dense(128, activation='relu')(d3)
    d5 = Dense(64, activation='relu')(d4)
    output = Dense(num_classes, activation='softmax')(d5)
    model = Model(inputs=pretrained.input, outputs=output)
    return model




In [73]:
face_model = build_model(vgg)

In [74]:
face_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [75]:
face_model.output_shape

(None, 4)

In [76]:
import tensorflow as tf

In [77]:
face_model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001), loss= tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [78]:
train_path = 'images/train_set/'
valid_path = 'images/validation_set/'



In [79]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

In [80]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2)


In [81]:
train_datagen

In [82]:
train_set = train_datagen.flow_from_directory(train_path, target_size=(image_width, image_height), batch_size=1, class_mode='categorical')


Found 120 images belonging to 4 classes.


In [83]:
valid_set = train_datagen.flow_from_directory(valid_path, target_size=(image_width, image_height), batch_size=1, class_mode='categorical')

Found 40 images belonging to 4 classes.


In [84]:
len(train_set)

120

In [85]:
len(valid_set)

40

In [86]:
res = face_model.fit_generator(train_set, validation_data=valid_set, epochs=3, steps_per_epoch=len(train_set), validation_steps=len(valid_set), verbose=1)

Epoch 1/3


C:\Users\Ponda\AppData\Local\Temp\ipykernel_5344\3783086810.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  res = face_model.fit_generator(train_set, validation_data=valid_set, epochs=3, steps_per_epoch=len(train_set), validation_steps=len(valid_set), verbose=1)


 86/120 [====================>.........] - ETA: 22s - loss: 3.0428 - accuracy: 0.4419

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(res.history['loss'], label='train loss')
plt.plot(res.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

plt.plot(res.history['accuracy'], label='train acc')
plt.plot(res.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')